In [1]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.notebook import trange, tqdm, tqdm_notebook
from joblib import Parallel, delayed
import re


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()
data_processed = data_dir / 'processed'
data_raw = r'Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\PLA\HCC827-derived OCT mouse'


# Get info

Here we look at stitched images in all z stacks

In [4]:
markers_map = {
    'cycle1': {
        1: 'Hoeschst', 
        4: 'TEAD1 & YAP1'
    },
    'cycle2': {
        1: 'Hoeschst', 
        4: 'CylinE & CDK2'
    },
    'cycle3': {
        1: 'Hoeschst', 
        4: 'P-ERK & c-MYC'
    },
    'cycle4': {
        1: 'Hoeschst', 
        4: 'p-AKT & mTOR'
    },
    'cycle5': {
        1: 'Hoeschst', 
        4: 'Mcl-1 & BAK'
    },
    'cycle6': {
        1: 'Hoeschst',
        2: 'p-EGFR',
        3: 'Tom20',
        4: 'Ki67'
    },
    'cycle7': {
        1: 'Hoeschst',
        2: 'Pan-cytokeratin',
        3: 'Golph4',
        4: 'Bim'
    },
    'cycle8': {
        1: 'Hoeschst',
        2: 'Concanavalin A',
        3: 'Phalloidin',
        4: 'WGA'
    },
    'cycle9': {
        1: 'Hoeschst',
        2: 'NBD-C6'
    },
}

def get_info(data_raw, marker_dict = markers_map):
    timepoints = []
    resolutions = []
    fovs = []
    cycles = []
    afs = []
    channels = []
    markers = []
    paths = [] 
    
    # Loop through image folder
    for (dirpath, dirnames, filenames) in os.walk(data_raw):
        for name in sorted(filenames):
            if "tif" in name and "stitched" in name and 'defocused' not in dirpath:
                # Get information from image name
                d_split = dirpath.split('\\')
                n_split = name.split('_')
                                
                time = d_split[-1].split('_')[0]
                fov = d_split[-1].split('_')[-1]
                if 'FW' not in fov:
                    res = '20X'
                    fov = ''
                else:
                    res = '40X'
                    
                cycle = d_split[-1].split('_')[1]
                if 'Af' in cycle:
                    after_bleach = True
                    cycle = cycle[2:]
                else:
                    after_bleach = False
                
                ch = int(n_split[1][0])
                try:
                    marker = marker_dict[cycle][ch]
                except:
                    continue 
                    
                timepoints.append(time)
                resolutions.append(res)
                fovs.append(fov)
                cycles.append(cycle)
                afs.append(after_bleach)
                channels.append(ch)
                markers.append(marker)
                paths.append(os.path.join(dirpath, name))
                
    info = {
            "Timepoint": timepoints,
            "Resolution": resolutions,
            "FOV": fovs,
            "Cycle": cycles,
            "AfBleach": afs,
            "Channels": channels,
            "Markers": markers,
            "Path": paths
        }

    df = pd.DataFrame(info)
    return df

In [5]:
df_meta_path = data_dir / 'OCT mouse' / 'Whole' / 'metadata' / 'info.csv'

try:
    df_meta_path.parent.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")

df_exist = df_meta_path.is_file()

if not df_exist:
    print('Created df')
    df = get_info(data_raw)
    df.to_csv(df_meta_path, index=False)
else:
    print('Loaded df')
    df = pd.read_csv(df_meta_path)

Folder is already there
Loaded df


In [6]:
df

,Timepoint,Resolution,FOV,Cycle,AfBleach,Channels,Markers,Path
0,1M,20X,NaN,cycle1,True,1,Hoeschst,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
1,1M,20X,NaN,cycle1,True,4,TEAD1 & YAP1,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
2,1M,40X,FW1,cycle1,True,1,Hoeschst,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
3,1M,40X,FW1,cycle1,True,4,TEAD1 & YAP1,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
4,1M,40X,FW2,cycle1,True,1,Hoeschst,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
...,...,...,...,...,...,...,...,...
330,1W,40X,FW1,cycle9,False,2,NBD-C6,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
331,1W,40X,FW2,cycle9,False,1,Hoeschst,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
332,1W,40X,FW2,cycle9,False,2,NBD-C6,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
333,1W,40X,FW3,cycle9,False,1,Hoeschst,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."


# Convert data to hdf5 

Convert stitch data to hdf5 format.

For each file we are organized into the format of: File -> Cycle 

Attributes are Channels and Markers

In [7]:
import h5py

def save_hdf5(
    path: str, name: str, data: np.ndarray, attr_dict=None, mode: str = "a"
) -> None:
    # Read h5 file
    hf = h5py.File(path, mode)
    # Create z_stack_dataset
    if hf.get(name) is None:
        data_shape = data.shape
        data_type = data.dtype
        chunk_shape = (1,) + data_shape[1:]
        max_shape = (data_shape[0],) + data_shape[1:]
        dset = hf.create_dataset(
            name,
            shape=data_shape,
            maxshape=max_shape,
            chunks=chunk_shape,
            dtype=data_type,
            compression="gzip",
        )
        dset[:] = data
        if attr_dict is not None:
            for attr_key, attr_val in attr_dict.items():
                dset.attrs[attr_key] = attr_val
    else:
        print(f"Dataset {name} exists")

    hf.close()

def read_img(path):
    return skimage.io.imread(path, as_gray=True)

def joblib_loop(task, pics):
    return Parallel(n_jobs=20)(delayed(task)(i) for i in pics)

In [8]:
df_imgs_path = data_dir / 'OCT mouse' / 'Whole' / 'metadata' / 'imgs.csv'

try:
    df_imgs_path.parent.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")
    
temp_path = data_dir / 'OCT mouse' / 'Whole' / 'hdf5' / 'raw'
try:
    temp_path.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")

df_exist = df_imgs_path.is_file()

if not df_exist:
    print('Created df')
    df_z = df[df.Resolution == '40X']
    group = df_z.groupby(['Timepoint', 'Resolution', 'FOV', 'AfBleach'])
    rows = []

    for name, df_group in tqdm(group, total=len(group)):
        file_name = '_'.join(np.array(name).astype(str)) + '.hdf5'
        file_path = temp_path / file_name
        rows.append(list(name)+[file_path])
        
        group_cycle = df_group.groupby('Cycle')
        for cycle, df_cycle in group_cycle:
            channels = df_cycle.Channels.to_list()
            markers = df_cycle.Markers.to_list()
            paths = df_cycle.Path.to_numpy()
    
            imgs = joblib_loop(read_img, paths)
            imgs = np.array(imgs)
            info = {"Channels": channels, "Markers": markers}
            
            # hdf5 as Channel -> Z mapping
            save_hdf5(file_path, cycle, imgs, info)
    df_imgs = pd.DataFrame(rows, columns=['Timepoint', 'Resolution', 'FOV', 'AfBleach', 'Path'])        
    df_imgs.to_csv(df_imgs_path, index=False)
else:
    print('Loaded df')
    df_imgs = pd.read_csv(df_imgs_path)

Folder is already there
Folder is already there
Loaded df


In [9]:
df_imgs = df_imgs[df_imgs.AfBleach == False]

# Get shift per cycle

In [53]:
from skimage.registration import phase_cross_correlation
from IPython.display import display, clear_output
from skimage import exposure
import scipy

def contrast_str(img):
    p2, p98 = np.percentile(img, (0.1, 99.99))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))
    return img_rescale

def whiten(img, sigma):
    img = skimage.img_as_float32(img)
    if sigma == 0:
        output = scipy.ndimage.convolve(img, _laplace_kernel)
    else:
        output = scipy.ndimage.gaussian_laplace(img, sigma)
    return output

def register(img1, img2, sigma, upsample=1):
    img1w = whiten(img1, sigma)
    img2w = whiten(img2, sigma)
    shift = skimage.registration.phase_cross_correlation(
        img1w,
        img2w,
        upsample_factor=upsample,
        normalization=None,
        return_error=False,
    )
    # At this point we may have a shift in the wrong quadrant since the FFT
    # assumes the signal is periodic. We test all four possibilities and return
    # the shift that gives the highest direct correlation (sum of products).
    shape = np.array(img1.shape)
    shift_pos = (shift + shape) % shape
    shift_neg = shift_pos - shape
    shifts = list(itertools.product(*zip(shift_pos, shift_neg)))
    correlations = [
        np.abs(np.sum(img1w * scipy.ndimage.shift(img2w, s, order=0)))
        for s in shifts
    ]
    idx = np.argmax(correlations)
    shift = shifts[idx]
    correlation = correlations[idx]
    total_amplitude = np.linalg.norm(img1w) * np.linalg.norm(img2w)
    if correlation > 0 and total_amplitude > 0:
        error = -np.log(correlation / total_amplitude)
    else:
        error = np.inf
    return shift, error

def make_imgs_same_dim(imgs):
    # Get max dimensions
    shapes = np.array([img.shape for img in imgs])
    min_x, min_y = shapes.min(axis=0)
        
    return [img[:min_x, :min_y] for img in imgs]

def get_shift_between_cycle(imgs, cycles, sigma=1, upsample=1):
    """Function to get shift within each cycle for the DAPI channel

    Args:
        df (pd DataFrame) : info dataframe for images in dapi channel accross cycle

    Returns:
        shift dictionnary
    """
    shift_dict = {}
    
    # Max shift accross all cycle:
    max_shift_x = 0
    min_shift_x = 0
    max_shift_y = 0
    min_shift_y = 0
    reference_dapi = imgs[0]
    cycles_it = cycles[1:]
    for i, img_dapi in enumerate(imgs[1:]):
        
        # print(f'Registration cycle {cycles[i+1]}', flush=True)
        # Get image shift y and x and save to shift_dict
        try:
            # shift, _, _ = phase_cross_correlation(
            #     reference_dapi, img_dapi, 
            #     upsample_factor=1
            # )  # Shift vector required to register moving images with reference images. Axis orderingis constitent with Y,X
            shift, _ = register(reference_dapi, img_dapi, sigma, upsample=upsample)
        except ValueError as err:
            print(err)
        shift_y, shift_x = shift[0], shift[1]
        shift_dict[cycles[i+1]] = {"shift_x": shift_x, "shift_y": shift_y}

        # Update max shift
        max_shift_x = shift_x if shift_x > max_shift_x else max_shift_x
        min_shift_x = shift_x if shift_x < min_shift_x else min_shift_x
        max_shift_y = shift_y if shift_y > max_shift_y else max_shift_y
        min_shift_y = shift_y if shift_y < min_shift_y else min_shift_y

    max_shift_x = int(max_shift_x)
    min_shift_x = int(min_shift_x)
    max_shift_y = int(max_shift_y)
    min_shift_y = int(min_shift_y)

    shift_dict["max"] = {
        "shift_x": max_shift_x,
        "shift_y": max_shift_y,
    }
    shift_dict["min"] = {
        "shift_x": min_shift_x,
        "shift_y": min_shift_y,
    }

    return shift_dict

In [54]:
shift_csv = data_dir / 'OCT mouse' /  'Whole'/  'metadata' / "shift.csv"

try:
    shift_csv.parent.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")

if not shift_csv.is_file():
    
    dfs_shift = []
    for row in tqdm(df_imgs.itertuples(), total=len(df_imgs)):
        path = row.Path
        imgs_dapi = []
        cycles = []
        with h5py.File(path, "r") as f:
            for k in f.keys():
                channels = f[k].attrs['Channels']
                img = contrast_str(f[k][0])
                imgs_dapi.append(img)
                cycles.append(k[-1])
                
        imgs_dapi_same_shape = make_imgs_same_dim(imgs_dapi)
        shift = get_shift_between_cycle(imgs_dapi_same_shape, cycles)
        df_temp = pd.DataFrame(shift)
        old_idx = df_temp.index.to_frame() # Convert index to dataframe
        old_idx.insert(0, 'FOV', row.FOV)
        old_idx.insert(0, 'Timepoint', row.Timepoint)
        df_temp.index = pd.MultiIndex.from_frame(old_idx) # Convert back to MultiIndex
        dfs_shift.append(df_temp)
        
    df_shift = pd.concat(dfs_shift)
    df_shift.to_csv(shift_csv , index=True)
else:
    df_shift = pd.read_csv(shift_csv)

Folder is already there


  0%|          | 0/5 [00:00<?, ?it/s]

In [55]:
df_shift


2      3      4       5       6       7       8  \
Timepoint FOV 0                                                              
1M        FW1 shift_x  167.0    0.0   90.0   -26.0   -84.0  -122.0  -105.0   
              shift_y    0.0  128.0    0.0    65.0   -38.0    67.0   228.0   
          FW2 shift_x -259.0  -51.0  -58.0   -35.0   -47.0   -43.0   -18.0   
              shift_y  -52.0  -93.0 -104.0   -52.0    -1.0    37.0     0.0   
1W        FW1 shift_x  225.0  149.0 -256.0   204.0   141.0   101.0   -31.0   
              shift_y -135.0   36.0  157.0    31.0    36.0   207.0   164.0   
          FW2 shift_x   37.0 -142.0  144.0  -102.0  -435.0   393.0   -45.0   
              shift_y  -72.0  -69.0   -1.0    47.0   222.0    43.0   163.0   
          FW3 shift_x  393.0  831.0  802.0  1175.0  1300.0  1697.0  1312.0   
              shift_y  -19.0    8.0   77.0   -83.0    56.0   -71.0  -106.0   

                            9   max  min  
Timepoint FOV 0                           
1M        FW1 shift_x    26.0   167 -122  
              shift_y   -29.0   228  -38  
          FW2 shift_x     0.0     0 -259  
              shift_y     0.0    37 -104  
1W        FW1 shift_x   -58.0   225 -256  
              shift_y   -89.0   207 -135  
          FW2 shift_x   378.0   393 -435  
              shift_y   100.0   222  -72  
          FW3 shift_x  1339.0  1697    0  
              shift_y   -53.0    77 -106

### Test

In [23]:
df_subset = df_imgs[(df_imgs.Timepoint == '1M') & (df_imgs.Resolution == '40X') & (df_imgs.FOV == 'FW1') & (df_imgs.AfBleach == False)]


In [27]:
path = df_subset.Path.item()
imgs_dapi = []
cycles = []
with h5py.File(path, "r") as f:
    for k in f.keys():
        channels = f[k].attrs['Channels']
        img = contrast_str(f[k][0])
        imgs_dapi.append(img)
        cycles.append(k[-1])

In [28]:
imgs_dapi_same_shape =  make_imgs_same_dim(imgs_dapi)
shift = get_shift_between_cycle(imgs_dapi_same_shape, cycles)

In [30]:
shift

{'2': {'shift_x': 167.0, 'shift_y': 0.0},
 '3': {'shift_x': 0.0, 'shift_y': 128.0},
 '4': {'shift_x': 90.0, 'shift_y': 0.0},
 '5': {'shift_x': -26.0, 'shift_y': 65.0},
 '6': {'shift_x': -84.0, 'shift_y': -37.0},
 '7': {'shift_x': -122.0, 'shift_y': 67.0},
 '8': {'shift_x': -106.0, 'shift_y': 228.0},
 '9': {'shift_x': -6.0, 'shift_y': 3437.0},
 'max': {'shift_x': 167, 'shift_y': 3437},
 'min': {'shift_x': -122, 'shift_y': -37}}

In [41]:
sigma = 1
img1 = imgs_dapi_same_shape[0]
img2 = imgs_dapi_same_shape[-1]
img1w = whiten(img1, sigma)
img2w = whiten(img2, sigma)
shift = skimage.registration.phase_cross_correlation(
    img1w,
    img2w,
    upsample_factor=1,
    normalization=None,
    return_error=False,
)
# At this point we may have a shift in the wrong quadrant since the FFT
# assumes the signal is periodic. We test all four possibilities and return
# the shift that gives the highest direct correlation (sum of products).
shape = np.array(img1.shape)
shift_pos = (shift + shape) % shape
shift_neg = shift_pos - shape
shifts = list(itertools.product(*zip(shift_pos, shift_neg)))
correlations = [
    np.abs(np.sum(img1w * scipy.ndimage.shift(img2w, s, order=0)))
    for s in shifts
]


In [42]:
shifts

[(7416.0, 26.0), (7416.0, -15303.0), (-29.0, 26.0), (-29.0, -15303.0)]

In [47]:
idx = np.argmax(correlations)
shift = shifts[idx]
correlation = correlations[idx]
total_amplitude = np.linalg.norm(img1w) * np.linalg.norm(img2w)

In [48]:
shift

(-29.0, 26.0)

In [45]:
shift_19, _, _ = phase_cross_correlation(imgs_dapi_same_shape[0], imgs_dapi_same_shape[-1], upsample_factor=1, overlap_ratio=0.001, normalization=None,)  # Shift vector required to register moving images with reference images. Axis orderingis constitent with Y,X
shift_19

array([-39.,  22.])

In [46]:
shift_19, _, _ = phase_cross_correlation(img1w, img2w, upsample_factor=1, overlap_ratio=0.001, normalization=None,)  # Shift vector required to register moving images with reference images. Axis orderingis constitent with Y,X
shift_19

array([-29.,  26.], dtype=float32)

In [31]:
import napari

napari.view_image(np.stack(imgs), channel_axis=0, name=cycles)

C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\napari\plugins\_plugin_manager.py:549: UserWarning: Plugin 'napari-crop' has already registered a function widget 'crop region' which has now been overwritten
  warn(message=warn_message)
C:\Users\thu71\.conda\envs\thomas-env\lib\site-packages\napari_tools_menu\__init__.py:168: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 3722.0, 7664.0), zoom=0.02144301650466436, angles=(0.0, 0.0, 90.0), perspective=0, interactive=True), cursor=Cursor(position=(1, 1), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 7445.0, 1.0), (0.0, 15329.0, 1.0)), current_step=(3722, 7664), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer '1' at 0x1a1eaf44f40>, <Image layer '2' at 0x1a1e5cfa8e0>, <Image layer '3' at 0x1a1eb001be0>, <Image layer '4' at 0x1a2004d9a90>, <Image layer '5' at 0x1a1ea4e5940>, <Image layer '6' at 0x1a1ea591fd0>, <Image layer '7' at 0x1a1ea62bf10>, <Image layer '8' at 0x1a1ea72bfa0>, <Image layer '9' at 0x1a1ea7c5d90>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), tex

# Get shift cropped images

In [56]:
from skimage import transform
from functools import partial
import multiprocessing
import itertools 

def flatten_array(array):
    '''
    Flatten list of numpy array to a single list
    '''
    list_list = list(map(list,array))
    return list(itertools.chain.from_iterable(list_list))
    
def make_imgs_same_dim(imgs):
    # Get max dimensions
    shapes = np.array([img[0].shape for img in imgs])
    min_x, min_y = shapes.min(axis=0)
    
    return [img[:, :min_x, :min_y] for img in imgs]

def crop_images(imgs, df_shift, cycles, upscale=1):
    # Get max shift values
    max_shift_x = df_shift.loc['shift_x', 'max']
    min_shift_x = df_shift.loc['shift_x', 'min']
    max_shift_y = df_shift.loc['shift_y', 'max']
    min_shift_y = df_shift.loc['shift_y', 'min']
    
    imgs_cropped = []
    for idx, img in enumerate(imgs):
        if cycles[idx] == '1':
            # Crop image
            res_cropped = img[:, max_shift_y:min_shift_y-1, max_shift_x:min_shift_x-1]
        else:
            #Apply shift
            shift_x, shift_y = (
                df_shift.loc['shift_x', cycles[idx]],
                df_shift.loc['shift_y', cycles[idx]],
            )
            # rows, cols = img.shape
            # M = np.float32([[1, 0, shift_x], [0, 1, shift_y]])
            # res = cv2.warpAffine(img, M, (cols, rows))
            shift_x = int(shift_x*upscale)
            shift_y = int(shift_y*upscale)
            
            t = transform.AffineTransform(translation=[-shift_x, -shift_y])
            f_partial = partial(transform.warp, inverse_map=t,mode='constant', cval=0, preserve_range=True)
            # res = transform.warp(img, t, mode='constant', cval=0, preserve_range=True)
            # res = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(f_partial)(i) for i in img)
            res = [f_partial(i) for i in img]
            res = np.array(res)
            
            # Crop image
            res_cropped = res[:, max_shift_y:min_shift_y-1, max_shift_x:min_shift_x-1]
            res_cropped.astype(img.dtype)
        imgs_cropped.append(res_cropped)
    return imgs_cropped

In [57]:
df_reg_path = data_dir / 'OCT mouse' / 'Whole' / 'metadata' / 'imgs_reg.csv'

try:
    df_reg_path.parent.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")
    
temp_path = data_dir / 'OCT mouse' / 'Whole' / 'hdf5' / 'registered'
try:
    temp_path.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")

df_exist = df_reg_path.is_file()

if not df_exist:
    df_reg = df_imgs.copy()
    for row in tqdm(df_imgs.itertuples(), total=len(df_imgs)):
        # Get Path
        path = row.Path
        file_name = str(row.Path).split('\\')[-1]
        file_path = temp_path / file_name
        
        # Check if file exist
        if file_path.is_file():
            print(file_name+' exists!')
            df_reg.loc[row.Index, 'Path'] = file_path
            continue
        
        # Query from blur dataframe 
        imgs = []
        cycles = []
        cycles_per_img = []
        channels = []
        markers = []
        with h5py.File(path, "r") as f:
            for k in f.keys():
                imgs.append(f[k][:])
                cycles.append(k[-1])
                channels.append(f[k].attrs['Channels'])
                markers.append(f[k].attrs['Markers'])
                cycles_per_img.extend([k[-1] for i in range(len(f[k].attrs['Channels']))])
        try:
            df_shift_roi = df_shift.loc[row[1], row[3]]
            imgs_same_shape = make_imgs_same_dim(imgs)
            imgs_cropped = crop_images(imgs_same_shape, df_shift_roi, np.array(cycles).astype(str))
        except Exception as e:
            print(row, str(e))
            continue
        info = {"Cycle": cycles_per_img, "Channel": flatten_array(channels), "Marker": flatten_array(markers)}
        
        imgs_stacked = np.vstack(imgs_cropped)
        
        # hdf5 as Channel -> Z mapping
        save_hdf5(file_path, 'imgs', imgs_stacked, info)
        df_reg.loc[row.Index, 'Path'] = file_path
        
    df_reg.to_csv(df_reg_path, index=False)
else:
    print('Loaded df')
    df_reg = pd.read_csv(df_reg_path)

Folder is already there


  0%|          | 0/5 [00:00<?, ?it/s]